# Downloading Simulation Data

## Simulation Access with SimIM

**SimIM** draws galaxy (and dark matter halo) properties from cosmological simulations. The first step to using **SimIM** is therefore to download the necessary subhalo catalogs. Currently, **SimIM** supports subhalo catalogs from [Illustris](https://www.illustris-project.org/) and [IllustrisTNG](https://www.tng-project.org/), along with the [UniverseMachine](https://www.peterbehroozi.com/data.html) halo catalogs for the BolshoiPlanck and MultiDark simulations. Details on each simulation can be found on the linked web pages and associated publications. Any work using **SimIM** should separately acknowledge and reference the simulation used.

## Downloading a Simulation

In this example and throughout the **SimIM** documentation we use TNG100-3-Dark simulation. This is a simulation box with 100 Mpc sides, a dark matter particle mass of $4\times10^8$ $M_\odot$, and no Baryonic physics. We use this simulation because the total data volume is relatively small (~3 GB compared to ~1 TB for the full physics, full resolution TNG300-1 simulation). For most scientific applications it is advisable to utilize a higher resolution simulation. However the smaller, low-resolution TNG100-3 can be downloaded, formatted, and loaded much more quickly, making it useful for demonstration and testing purposes.

We can start by loading the ``simim.siminterface`` module, which is used for downloading and interacting with simulation halo catalogs:

In [1]:
import simim.siminterface as sim

If this is your first time using **SimIM** on a machine, you may also need to run the following lines and follow the prompts to set a path in which to store downloaded simulation data.

In [4]:
from simim import setupsimim
setupsimim()

Please specify a path to save data directories.
Specifying no path will set the path to your home directory.
Files will be saved in /Users/keenan/Data/simim_resources


Now we can proceed to download the subhalo catalogs for our prefered simulation. Note that this is a large quantity of data and can take some time depending on your internet connection. This initial download is in the raw format provided by the original halo catalogs, we will translate it to SimIM's format in a second step.

For Illustris and IllustrisTNG data, you will need an API key from the Illustris/TNG project, which can be obtained [here](https://www.tng-project.org/users/register/). For UniverseMachine catalogs no key is required.

In [6]:
# download the data - set the redownload parameter to True
# if you want to overwrite snapshots already saved on your 
# machine.

api_key = '[fill in your API key here]' # Replace this with a string containing key
cat = sim.IllustrisCatalogs('TNG100-3-Dark',api_key)
cat.download_meta()
cat.download()

2024-02-29 01:03:23 URL:http://www.tng-project.org/api/TNG100-3-Dark/files/groupcat-0/?format=api [9151/9151] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_000/index.html?format=api.tmp" [1]
2024-02-29 01:03:23 URL:http://www.tng-project.org/ [13013/13013] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_000/index.html.tmp" [1]
2024-02-29 01:03:23 URL:http://www.tng-project.org/about/ [36929/36929] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_000/index.html.tmp" [1]
2024-02-29 01:03:23 URL:http://www.tng-project.org/people/ [13268/13268] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_000/index.html.tmp" [1]
2024-02-29 01:03:23 URL:http://www.tng-project.org/results/ [484900/484900] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_000/index.html.tmp" [1]
2024-02-29 01:03:23 URL:http://www.tng-project.org/media/ [59010/59010] -> "/Users/keenan/Da

2024-02-29 01:03:26 URL:http://www.tng-project.org/api/TNG100-3-Dark/files/groupcat-1/?format=api [9151/9151] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_001/index.html?format=api.tmp" [1]
2024-02-29 01:03:26 URL:http://www.tng-project.org/ [13013/13013] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_001/index.html.tmp" [1]
2024-02-29 01:03:26 URL:http://www.tng-project.org/about/ [36929/36929] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_001/index.html.tmp" [1]
2024-02-29 01:03:26 URL:http://www.tng-project.org/people/ [13274/13274] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_001/index.html.tmp" [1]
2024-02-29 01:03:26 URL:http://www.tng-project.org/results/ [484900/484900] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_001/index.html.tmp" [1]
2024-02-29 01:03:26 URL:http://www.tng-project.org/media/ [59010/59010] -> "/Users/keenan/Da

2024-02-29 01:03:29 URL:http://www.tng-project.org/api/TNG100-3-Dark/files/groupcat-2/?format=api [9151/9151] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_002/index.html?format=api.tmp" [1]
2024-02-29 01:03:29 URL:http://www.tng-project.org/ [13013/13013] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_002/index.html.tmp" [1]
2024-02-29 01:03:29 URL:http://www.tng-project.org/about/ [36929/36929] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_002/index.html.tmp" [1]
2024-02-29 01:03:29 URL:http://www.tng-project.org/people/ [13267/13267] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_002/index.html.tmp" [1]
2024-02-29 01:03:29 URL:http://www.tng-project.org/results/ [484900/484900] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_002/index.html.tmp" [1]
2024-02-29 01:03:29 URL:http://www.tng-project.org/media/ [59010/59010] -> "/Users/keenan/Da

2024-02-29 01:03:32 URL:http://www.tng-project.org/api/TNG100-3-Dark/files/groupcat-3/?format=api [9151/9151] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_003/index.html?format=api.tmp" [1]
2024-02-29 01:03:32 URL:http://www.tng-project.org/ [13013/13013] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_003/index.html.tmp" [1]
2024-02-29 01:03:32 URL:http://www.tng-project.org/about/ [36929/36929] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_003/index.html.tmp" [1]
2024-02-29 01:03:32 URL:http://www.tng-project.org/people/ [13273/13273] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_003/index.html.tmp" [1]
2024-02-29 01:03:32 URL:http://www.tng-project.org/results/ [484900/484900] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_003/index.html.tmp" [1]
2024-02-29 01:03:32 URL:http://www.tng-project.org/media/ [59010/59010] -> "/Users/keenan/Da

2024-02-29 01:03:34 URL:http://www.tng-project.org/api/TNG100-3-Dark/files/groupcat-4/?format=api [9151/9151] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_004/index.html?format=api.tmp" [1]
2024-02-29 01:03:34 URL:http://www.tng-project.org/ [13013/13013] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_004/index.html.tmp" [1]
2024-02-29 01:03:35 URL:http://www.tng-project.org/about/ [36929/36929] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_004/index.html.tmp" [1]
2024-02-29 01:03:35 URL:http://www.tng-project.org/people/ [13267/13267] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_004/index.html.tmp" [1]
2024-02-29 01:03:35 URL:http://www.tng-project.org/results/ [484900/484900] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_004/index.html.tmp" [1]
2024-02-29 01:03:35 URL:http://www.tng-project.org/media/ [59010/59010] -> "/Users/keenan/Da

...

2024-02-29 01:08:18 URL:http://www.tng-project.org/api/TNG100-3-Dark/files/groupcat-95/?format=api [9164/9164] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_095/index.html?format=api.tmp" [1]
2024-02-29 01:08:19 URL:http://www.tng-project.org/ [13013/13013] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_095/index.html.tmp" [1]
2024-02-29 01:08:19 URL:http://www.tng-project.org/about/ [36929/36929] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_095/index.html.tmp" [1]
2024-02-29 01:08:19 URL:http://www.tng-project.org/people/ [13274/13274] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_095/index.html.tmp" [1]
2024-02-29 01:08:19 URL:http://www.tng-project.org/results/ [484895/484895] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_095/index.html.tmp" [1]
2024-02-29 01:08:19 URL:http://www.tng-project.org/media/ [59010/59010] -> "/Users/keenan/D

2024-02-29 01:08:22 URL:http://www.tng-project.org/api/TNG100-3-Dark/files/groupcat-96/?format=api [9164/9164] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_096/index.html?format=api.tmp" [1]
2024-02-29 01:08:22 URL:http://www.tng-project.org/ [13013/13013] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_096/index.html.tmp" [1]
2024-02-29 01:08:22 URL:http://www.tng-project.org/about/ [36929/36929] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_096/index.html.tmp" [1]
2024-02-29 01:08:22 URL:http://www.tng-project.org/people/ [13269/13269] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_096/index.html.tmp" [1]
2024-02-29 01:08:22 URL:http://www.tng-project.org/results/ [484895/484895] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_096/index.html.tmp" [1]
2024-02-29 01:08:22 URL:http://www.tng-project.org/media/ [59010/59010] -> "/Users/keenan/D

2024-02-29 01:08:25 URL:http://www.tng-project.org/api/TNG100-3-Dark/files/groupcat-97/?format=api [9164/9164] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_097/index.html?format=api.tmp" [1]
2024-02-29 01:08:25 URL:http://www.tng-project.org/ [13013/13013] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_097/index.html.tmp" [1]
2024-02-29 01:08:25 URL:http://www.tng-project.org/about/ [36929/36929] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_097/index.html.tmp" [1]
2024-02-29 01:08:25 URL:http://www.tng-project.org/people/ [13267/13267] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_097/index.html.tmp" [1]
2024-02-29 01:08:25 URL:http://www.tng-project.org/results/ [484895/484895] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_097/index.html.tmp" [1]
2024-02-29 01:08:25 URL:http://www.tng-project.org/media/ [59010/59010] -> "/Users/keenan/D

2024-02-29 01:08:28 URL:http://www.tng-project.org/api/TNG100-3-Dark/files/groupcat-98/?format=api [9164/9164] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_098/index.html?format=api.tmp" [1]
2024-02-29 01:08:28 URL:http://www.tng-project.org/ [13013/13013] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_098/index.html.tmp" [1]
2024-02-29 01:08:28 URL:http://www.tng-project.org/about/ [36929/36929] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_098/index.html.tmp" [1]
2024-02-29 01:08:28 URL:http://www.tng-project.org/people/ [13267/13267] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_098/index.html.tmp" [1]
2024-02-29 01:08:28 URL:http://www.tng-project.org/results/ [484895/484895] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_098/index.html.tmp" [1]
2024-02-29 01:08:28 URL:http://www.tng-project.org/media/ [59010/59010] -> "/Users/keenan/D

2024-02-29 01:08:31 URL:http://www.tng-project.org/api/TNG100-3-Dark/files/groupcat-99/?format=api [9164/9164] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_099/index.html?format=api.tmp" [1]
2024-02-29 01:08:31 URL:http://www.tng-project.org/ [13013/13013] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_099/index.html.tmp" [1]
2024-02-29 01:08:31 URL:http://www.tng-project.org/about/ [36929/36929] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_099/index.html.tmp" [1]
2024-02-29 01:08:31 URL:http://www.tng-project.org/people/ [13274/13274] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_099/index.html.tmp" [1]
2024-02-29 01:08:31 URL:http://www.tng-project.org/results/ [484895/484895] -> "/Users/keenan/Data/simim_resources/simulations/TNG100-3-Dark/raw/groups_099/index.html.tmp" [1]
2024-02-29 01:08:31 URL:http://www.tng-project.org/media/ [59010/59010] -> "/Users/keenan/D

Now that the simulation is downloaded we can reformat it to use with **SimIM**. This procedure reformats the data in a uniform format used by **SimIM** to interface with all supported simulations.

Formatting is handled by the ``cat.format`` method. A few options are supported for formatting. Setting the ``basic`` parameter to ``True`` keeps only a minimal set of halo properties - position velocity, mass, radius, and a few others - which in the formatted catalog. This can be done to save space when other properties will not be needed.

Setting the ``realtime_clean_raw`` parameter to ``True`` will delete the unformatted data (which is kept on the disk by default) as the **SimIM** formatted file is generated. This can save a significant amount of disk space when working with larger simulations, but will require re-downloading data if something goes wrong. We can then also delete the data later, once we've verified that the catalog has been successfully converted to the **SimIM** format, and this approach should be prefered when disk space is not a concern.

In [7]:
cat.format(remake=True,basic=False,realtime_clean_raw=False)

Formatting snap 0
Formatting snap 1
Formatting snap 2
Formatting snap 3
Formatting snap 4
Formatting snap 5
Formatting snap 6
Formatting snap 7
Formatting snap 8
Formatting snap 9
Formatting snap 10
Formatting snap 11
Formatting snap 12
Formatting snap 13
Formatting snap 14
Formatting snap 15
Formatting snap 16
Formatting snap 17
Formatting snap 18
Formatting snap 19
Formatting snap 20
Formatting snap 21
Formatting snap 22
Formatting snap 23
Formatting snap 24
Formatting snap 25
Formatting snap 26
Formatting snap 27
Formatting snap 28
Formatting snap 29
Formatting snap 30
Formatting snap 31
Formatting snap 32
Formatting snap 33
Formatting snap 34
Formatting snap 35
Formatting snap 36
Formatting snap 37
Formatting snap 38
Formatting snap 39
Formatting snap 40
Formatting snap 41
Formatting snap 42
Formatting snap 43
Formatting snap 44
Formatting snap 45
Formatting snap 46
Formatting snap 47
Formatting snap 48
Formatting snap 49
Formatting snap 50
Formatting snap 51
Formatting snap 52
For

To remove the unformatted versions of the catalog, we can now call the ``cat.clean_raw`` method.

In [9]:
cat.clean_raw()

This will delete raw files for TNG100-3-Dark.
Aborting cleanup


## Partial Simulation Downloads

It is also possible to save time and disk space by only downloading and formatting snapshots you need. For instance, if we only wanted the simulation from redshifts 0 to 2 (snapshots 33-99 for IllustrisTNG), then we could do the following:

In [ ]:
snaps = np.arange(33,100)
api_key = '[fill in your API key here]' # Replace this with a string containing key
cat = sim.illustris.IllustrisCatalogs('TNG100-3-Dark',api_key,snaps=snaps)
cat.download_meta(redownload=True)
cat.download()
cat.format(remake=True)

Note that some of the metadata saved with the formatted snapshots depends on the full list of snapshots to be included in the **SimIM** catalogs. Therefore the ``cat.download_meta``, ``cat.download`` and ``cat.format`` methods should all be run with the a  ``simim.siminterface.IllustrisCatalogs`` instance initialized with the same snapshot list.

## Additional Simulations

All simulations in the Illustris and IllustrisTNG can be downloaded and formatted using the ``simim.siminterface.IllustrisCatalogs`` class specifying the name of the desired simulation as the ``sim`` argument. The available simulations are ``Illustris-[N]`` and ``Illustris-[N]-Dark`` (where [N] is replaced with 1, 2, or 3), and ``TNG[S]-[N]`` and ``TNG[S]-[N]-Dark`` (where [S] is 50, 100, or 300 and [N] is again 1, 2, or 3).

UniverseMachine catalogs for the BolshoiPlanck, MultiDark Planck 2, and Small MultiDark Placnk simulations. These can be accessed via ``simim.siminterface.UniversemachineCatalogs`` class and the names ``UniverseMachine-BolshoiPlanck``, ``UniverseMachine-MDPL2`` and ``UniverseMachine-SMDPL``.

It is also relatively straightforward to implement support for new simulations
within the **SimIM** framework. Doing so requires adding new code to the 
**SimIM** package, which can be added by cloning the source code from the
[SimIM project GitHub](https://github.com/rpkeenan/simim_public).
The procedure for implementing new simulations is then described in the
documentation for the ``simim.siminterface._rawsiminterface`` module.